# Few-Shot Demonstration Selection

Select the best example to use as a demonstatration in the few-prompt tests.

The goal is to find the document that not only contains a good quantity of the entity to be extracted but also a good divercity in the types of entties to be extracted. 

In [2]:
from collections import Counter
from pathlib import Path

import numpy as np

from src.reader import read_lusa
from src.utils import is_sublist
from src.prompts import OneShotPrompter

ROOT = Path().resolve().parent
DATA_PATH = ROOT / "resources" / "lusa"

In [3]:
lusa = read_lusa(DATA_PATH)

In [4]:
def get_best_document(documents, entities, attribute):
    best_doc = None
    max_n_entities, max_n_classes = None, None
    for doc in documents:
        
        doc_classes = set([
            getattr(entity, attribute)
            for entity in getattr(doc, entities)
            if hasattr(entity, str(attribute))
        ])
        
        n_doc_classes = len(doc_classes)
        n_entities = len(getattr(doc, entities))

        if best_doc is None:
            best_doc = doc
            max_n_entities = n_entities
            max_n_classes = n_doc_classes
            continue
        
        if n_doc_classes >= max_n_classes and n_entities >= max_n_entities:
            max_n_entities = n_entities
            max_n_classes = n_doc_classes
            best_doc = doc
    return best_doc

## Events

In [5]:
event_classes = set(event.class_ for doc in lusa for event in doc.events if hasattr(event, "class_"))
print("Event classes:", event_classes)

Event classes: {'State', 'Aspectual', 'Reporting', 'Occurrence', 'I_State', 'Perception', 'I_Action'}


In [6]:
n_event_per_doc = [len(doc.events) for doc in lusa] 
max_events = max(n_event_per_doc)

print("Max events per document:", max_events)

Max events per document: 46


In [7]:
max_events_idx = np.argmax(n_event_per_doc)
doc_max_events = lusa[max_events_idx]

max_doc_class_count = Counter([event.class_ for event in doc_max_events.events if hasattr(event, "class_")])
print("Max events per document class count:", max_doc_class_count)

Max events per document class count: Counter({'Occurrence': 20, 'State': 18, 'Reporting': 6, 'I_Action': 1, 'I_State': 1})


In [8]:
best_doc = get_best_document(lusa, "events", "class_")

In [9]:
print("Document with the most events and number of classes:", best_doc.id)

Document with the most events and number of classes: lusa_181


In [10]:
annotation = [ent.text for ent in best_doc.events]
prompter = OneShotPrompter(best_doc.text, annotation)
print(prompter.prompt)

Input: Covid-19: Polícia moçambicana detém homem indiciado por falsificação de testes
Maputo, 20 nov 2020 ( Lusa) - A Polícia da República de Moçambique (PRM) deteve um homem suspeito de falsificar testes de covid-19 com resultados negativos, anunciou hoje o porta-voz da corporação.
"Neste processo, nós conseguimos prender este indivíduo e apreender alguns testes falsos, que seriam enviados para pessoas que os teriam solicitado para fins de viagem", declarou Leonel Muchine, porta-voz da polícia moçambicana na cidade de Maputo.
O indiciado, que confessou o crime, trabalhava com outro indivíduo, que agora se encontra foragido, segundo as autoridades.
"Estamos a trabalhar para a localização do seu comparsa", afirmou o porta-voz da polícia.
O teste negativo de covid-19 é obrigatório para quem quer sair ou entrar no país e as autoridades têm reforçado a fiscalização nas fronteiras desde que o chefe de Estado, Filipe Nyusi, anunciou, em 29 de outubro, a retoma da emissão de vistos de turismo

## Time Expressions

In [11]:
timex_classes = set(timex.time_type for doc in lusa for timex in doc.timexs if hasattr(timex, "time_type"))
print("Timex classes:", timex_classes)

Timex classes: {'Date', 'Time', 'Set', 'Duration'}


In [12]:
n_timex_per_doc = [len(doc.timexs) for doc in lusa] 
max_timex = max(n_timex_per_doc)

print("Max events per document:", max_timex)

Max events per document: 9


In [13]:
max_timex_idx = np.argmax(n_timex_per_doc)
doc_max_timexs = lusa[max_timex_idx]

max_doc_class_count = Counter([timex.time_type for timex in doc_max_timexs.timexs if hasattr(timex, "time_type")])
print("Max events per document class count:", max_doc_class_count)

Max events per document class count: Counter({'Time': 4, 'Date': 3, 'Duration': 2})


In [14]:
best_doc = get_best_document(lusa, "timexs", "time_type")
print("Document with the most timexs and number of classes:", best_doc.id)
print("Number of timexs:", len(best_doc.timexs))
print("Number of timex classes:", len(set([timex.time_type for timex in best_doc.timexs if hasattr(timex, "time_type")])))

Document with the most timexs and number of classes: lusa_11
Number of timexs: 9
Number of timex classes: 3


In [15]:
annotation = [ent.text for ent in best_doc.timexs]
prompter = OneShotPrompter(best_doc.text, annotation)
print(prompter.prompt)

Input: Autoridades moçambicanas apreendem mais de uma tonelada de caranguejo
A fiscalização marítima moçambicana apreendeu 1.100 quilos de caranguejo, no centro do país, em menos de uma semana, capturado na "época de veda", quando é proibido apanhar a espécie, disse hoje à Lusa fonte das autoridades.
A última apreensão aconteceu no sábado quando as autoridades descobriram uma embarcação com 600 quilos de caranguejo.
“Estamos a apreender caranguejo e embarcações e os responsáveis incorrem em pesadas multas, caso sejam neutralizados”, explicou o chefe da fiscalização, César Maphossa.
No sábado, os tripulantes abandonaram o barco, fundeado nos arredores da cidade da Beira, centro de Moçambique, quando se aperceberam da chegada dos fiscais.
A embarcação foi confiscada e os caranguejos, dissimulados em caixas, foram posteriormente devolvidos ao seu habitat natural, no mangal do rio Maria, arredores da capital provincial de Sofala.
A apreensão ocorreu menos de uma semana após as autoridades 

## Participants

In [16]:
participant_classes = set(participant.participant_type_domain for doc in lusa for participant in doc.participants if hasattr(participant, "participant_type_domain"))
print("Participant classes:", participant_classes)

Participant classes: {'Other', 'Fac', 'Org', 'Nat', 'Obj', 'Per', 'Loc'}


In [17]:
n_part_per_doc = [len(doc.participants) for doc in lusa] 
max_part = max(n_part_per_doc)

print("Max events per document:", max_part)

Max events per document: 48


In [18]:
max_part_idx = np.argmax(n_part_per_doc)
doc_max_parts = lusa[max_part_idx]

max_doc_class_count = Counter([part.participant_type_domain for part in doc_max_parts.participants if hasattr(part, "participant_type_domain")])
print("Max events per document class count:", max_doc_class_count)

Max events per document class count: Counter({'Per': 17, 'Loc': 15, 'Org': 8, 'Nat': 6, 'Other': 2})


In [19]:
best_doc = get_best_document(lusa, "participants", "participant_type_domain")
print("Document with the most participants and number of classes:", best_doc.id)
print("Number of participants:", len(best_doc.participants))
print("Number of participants classes:", len(set([part.participant_type_domain for part in best_doc.participants if hasattr(part, "participant_type_domain")])))

Document with the most participants and number of classes: lusa_157
Number of participants: 47
Number of participants classes: 6


In [21]:
annotation = [ent.text for ent in best_doc.participants]
prompter = OneShotPrompter(best_doc.text, annotation)
print(prompter.prompt)

Input: Homem armado faz vários reféns dentro de um banco na Geórgia
Um homem armado fez hoje à tarde vários reféns, ainda em número incerto, dentro das instalações de um banco na Geórgia, informaram as autoridades desta ex-república soviética.
O Ministério do Interior da Geórgia não precisou, até ao momento, quantas pessoas foram feitas reféns dentro do banco, localizado na cidade de Zugdidi (região oeste), ou quais são as exigências do agressor.
A polícia isolou, entretanto, a zona onde fica a sucursal bancária e montou uma operação "para neutralizar o agressor", informou o ministério num comunicado.
A televisão estatal da Geórgia, a Mtavari TV, noticiou que o sequestrador está armado com uma granada de mão e exige 500.000 dólares (cerca de 420.000 euros) em dinheiro.
A Mtavari TV conseguiu falar com um dos reféns que indicou que o agressor mantém 19 pessoas dentro das instalações bancárias.
O canal de televisão também divulgou um vídeo que mostra uma sala com pessoas sentadas no chão